In [50]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import re
import csv
import time

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

url = 'https://companiesmarketcap.com'

In [51]:
def get_earnings(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Earnings
    earnings = dom.xpath('//h2/strong/span')
    if earnings:
        return earnings[0].text
    else:
        return None
    
    
def get_revenue(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Earnings
    revenue = dom.xpath('//h2/strong/span')
    if revenue:
        return revenue[0].text
    else:
        return None

def get_shares(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Earnings
    shares = dom.xpath('//h2/strong/span')
    if shares:
        shares_txt = shares[0].text
        shares_txt = shares_txt.replace(',', '.')
        return shares_txt
    else:
        return None
    
def save_category(cat, id_azienda, writer):
    cat_name = re.sub(r'[^a-zA-Z0-9]', '', cat)
    row = [id_azienda, cat_name, hash(cat_name)]    
    writer.writerow(row)

In [52]:
def get_info_marketcap(url2, employees, id_, writer_a, writer_c):
    req = requests.get(url2, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    global shares2
    
    #Nome
    name = dom.xpath('//div[@class = "company-name"]')
    if name:
        name_txt = name[0].text
        name_txt = name_txt.replace('"', '')
        name_txt = name_txt.replace('\n', '')
    else:
        name_txt = None
    
    #Marketcap
    mc = dom.xpath('//div[text() = "Marketcap"]/../div[1]')
    if mc:
        mc_txt = mc[0].text
    else:
        mc_txt = none
    
    #Share price
    sp = dom.xpath('//div[text() = "Share price"]/../div[1]')
    if sp:
        sp_txt = sp[0].text
    else:
        sp = None
    
    #Company code
    cc = dom.xpath('//div[@class = "company-code"]')
    if cc:
        cc_txt = cc[0].text
    else:
        cc_txt = None
        
    #Revenue e Earnings
    links_1 = dom.xpath('//a[@class = "nav-link"]/@href')
    for link in links_1:
        if '/earnings' in link:
            earnings = get_earnings(url + link)
        elif 'revenue' in link:
            revenue = get_revenue(url + link)
    
    #Number of shares
    links_2 = dom.xpath('//div[@class = "dropdown-menu"]/a/@href')
    for link in links_2:
        if '/shares-outstanding' in link:
            shares2 = get_shares(url + link)
            
    #Categories
    categories = dom.xpath('//*[@id="cmkt"]/div[3]/div[1]/div[2]/div[3]/div[1]/a/text()')
    for cat in categories:
        save_category(cat, id_, writer_c)
    
    
    employees = employees.replace(',', '.')
    
    row = [url2, id_, name_txt, cc_txt, mc_txt, sp_txt, earnings, revenue, shares2, employees]
    writer_a.writerow(row)

In [53]:
ds_a = open('cmc-aziende-dataset.csv', 'w')
head_a = ['URL', 'ID', 'Name', 'Company code', 'Marketcap', 'Share price', 
        'Earnings', 'Revenue', 'Shares','Employees']
writer_a = csv.writer(ds_a)
writer_a.writerow(head_a)

ds_c = open('cmc-categorie-dataset.csv', 'w')
head_c = ['Company ID', 'Category', 'Hash']
writer_c = csv.writer(ds_c)
writer_c.writerow(head_c)

id_ = 0
start = time.time()
for x in range(1, 15):     #(1, 15)
    req = requests.get(url + '/usa/largest-american-companies-by-number-of-employees/?page=' + str(x), headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    links = dom.xpath('//a[contains(@href, "/marketcap")]/@href')
    employees = dom.xpath('//td[3]')
    for n in range(0, 100):   # (0, 100)
        id_ += 1
        url2 = url + links[n]
        get_info_marketcap(url2, employees[n].text, id_, writer_a, writer_c)
end = time.time();
ds_a.close()
ds_c.close()
print(end - start)
print(id_, 'aziende')

Walmart
Amazon
United Parcel Service
Kroger
Home Depot
Target
Starbucks
Berkshire Hathaway 
UnitedHealth
FedEx
Cognizant Technology Solutions 
TJX Companies
Pepsico
Costco
Lowe's Companies
Concentrix
JPMorgan Chase
IBM
Jabil
Aramark
Wells Fargo
Citigroup
Microsoft
CVS Health
Bank of America 
HCA Healthcare
AT&T
Walgreens Boots Alliance
Comcast
Alphabet (Google)
Ford
Darden Restaurants
Raytheon Technologies
General Electric
Apple
Dollar General
Lear Corporation
General Motors
Walt Disney
Oracle
Hilton Worldwide
Boeing
Johnson & Johnson
Tyson Foods
Dell
DXC Technology
Thermo Fisher Scientific
American Airlines
ABM Industries
Barrett Business Services
Intel
GXO Logistics
Marriott International
Verizon
Lockheed Martin
Abbott Laboratories
Caterpillar
Procter & Gamble
CBRE Group
General Dynamics
Albertsons
Ross Stores
Chipotle Mexican Grill
McDonald
Tesla
Honeywell
Kohl's
Elevance Health
Jones Lang LaSalle
Gap Inc.
Humana
3M
Charter Communications
Schlumberger
United Airlines Holdings
Kyndry

Modine Manufacturing
Valmont Industries
Coty
Warner Bros. Discovery
Carlisle Companies
Crane Co.
Lennox
Vivint Smart Home
Lincoln Electric
Matthews International Corporation
Patrick Industries
Skyworks Solutions
The Toro Company
Ingles Markets
Benchmark Electronics
Lincoln National Corporation
Primoris Services Corporation
Herbalife
SP Plus Corporation
eBay
Korn Ferry
Lennar
Post Holdings
Regeneron Pharmaceuticals
Juniper Networks
AdaptHealth
Steel Dynamics
Middleby
Fox Corporation
Kinder Morgan
Ally
Big Lots
Hillenbrand
Western Union
Rivian
PriceSmart
CoreCivic
Loews Corporation
The Greenbrier Companies
Masonite
Kemper
DTE Energy
Sonic Automotive
TransUnion
Live Nation
Unum
West Pharmaceutical
Ebix
First Watch Restaurant
Enhabit
Bally's Corporation
TreeHouse Foods
Tupperware Brands
Advanced Energy
American Woodmark
Visteon
Constellation Brands
Martin Marietta
IDEXX Laboratories
Zions Bancorporation
Driven Brands
ITT
Darling Ingredients
Rithm Capital
Spirit Airlines
Valvoline
Zebra Tec

Prosperity Bancshares
Audacy
Alpha Metallurgical Resources
UMB Financial
Paysafe
The Children's Place
Evolent Health
Reinsurance Group of America
Conn's
TriMas
TriNet
Cresco Labs
StoneX Group
CME Group
Schnitzer Steel
Utz Brands
Monster Beverage
Materion
PBF Energy
Atkore
DraftKings
Sonida Senior Living
Robinhood
CNO Financial Group
Allison Transmission
Mueller Water Products
Trinseo
BRP Group
Glacier Bancorp
Guidewire Software
Valley Bank
Drive Shack
Western Alliance Bancorporation
FICO
The ONE Group Hospitality
PRA Group
Dorman Products
The RealReal
Lamar Advertising
NexTier Oilfield
Alliant Energy
Delek US
The Shyft Group
Arch Resources
Denny's
Sportsman's Warehouse
Kratos Defense & Security Solutions
Smartsheet
Sharecare
Palantir
Glatfelter
Sprinklr
KKR & Co.
Infinera
Vornado Realty Trust
Columbus McKinnon
Globe Life
Essential Utilities
Simmons First National
Bright Health
Datadog
Fulton Financial
Homepoint
Consolidated Communications
SmileDirectClub
Lancaster Colony Corporation
Pi